In [1]:
import numpy as np

## Estudo da função

$$
\lim_{x1 \to 0} f(x1) = \infty \text{  e  } \lim_{x2 \to 0} f(x2) = \infty
$$

$$
f(x) = 3^{\frac{1}{x_1+x_2}} + x_1^2 + x_2^2 \\
\nabla f(x) = [2 \dot x_1 -\frac{ e^\frac{ln(3)}{x_1+x_2}}{(x_1+x_2)**2}, 2 \dot x_2 -\frac{ e^\frac{ln(3)}{x_1+x_2}}{(x_1+x_2)**2}] \\

$$

In [202]:
def f(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return 3**(1/(x1+x2))+x1**2+x2**2

In [203]:
def Gradiente(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return np.array([[2*x1-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2],
            [2*x2-(np.log(3)*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**2]])

In [204]:
def Heissiana(x):
    x = np.array(x)
    x1 = x[0][0]
    x2 =x[1][0]
    return np.array([[2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))))/(x1+x2)**4, -np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4],

            [-np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2)))/(x1+x2)**4 ,2 - (np.log(3)*(-np.log(3)*np.exp(np.log(3)/(x1+x2))-2*np.exp(np.log(3)/(x1+x2))*(x1+x2)))/(x1+x2)**4]])

In [116]:
def definida_positiva_2x2(matrix):    
    if matrix[0][0] > 0 and np.linalg.det(matrix)>0:
        return True
    else: 
        return False

## Implementação dos métodos

### Busca de Armijo

In [166]:
def Armijo(x,gama,d,n):
    t = 1
    k = 0
    d = np.array(d)
    x = np.array(x)
    while f(x + t*d) > f(x) + n*t*(np.transpose(Gradiente(x))@d):
        t=gama*t
        k = k+1
    return [t,k]

In [120]:
x = [[0.5],[0.5]]
n = 0.25
t = 1
d = [[3],[1]]

### Método do Gradiente

In [169]:
def met_gradiente(x,tol,maxiter,gama,n):
    x = np.array(x)
    m = 0
    k = 0
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        d = - Gradiente(x)
        ar = Armijo(x,gama,d,n)
        k = k + ar[1]
        t = ar[0]
        x = x + t*d
        m = m + 1
    return [x,m,k] 

In [170]:
met_gradiente([[-1],[-1]],0.01,1000,0.8,0.25)

C:\Users\yfrom\AppData\Local\Temp\ipykernel_17260\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


[array([[-0.00150145],
        [-0.00150145]]),
 3,
 14]

O método segue o gradiente até o ponto não definido da função (0,0).

In [172]:
met_gradiente([[0.5],[0.5]],0.00001,1000,0.8,0.25)

[array([[0.67633305],
       [0.67633305]]), 11, 89]


### Método de Newton

In [194]:
def met_newton(x,tol,maxiter,gama,n):
    x = np.array(x)
    m = 0
    k = 0
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        d = - (np.linalg.inv(Heissiana(x)))@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        k = k + ar[1]
        t = ar[0]
        x = x + t*d
        m = m + 1
        k = k+1
    return [x,m,k] 

In [177]:
met_newton([[-1],[-1]],0.01,1000,0.8,0.25) 

[array([[-0.06787345],
        [-0.1419706 ]]),
 1000,
 153962]

In [195]:
met_newton([[0.5],[0.5]],0.01,1000,0.8,0.25) 

[array([[0.678713  ],
        [0.67495778]]),
 3,
 3]

### Método Quase Newton

#### DFP

In [221]:
def DFP(x_atual,x_antigo,H):
    x_atual = np.array(x_atual)
    x_antigo = np.array(x_antigo)
    p = x_atual - x_antigo
    q = Gradiente(x_atual)- Gradiente(x_antigo)
    if np.linalg.norm(p) != 0 and np.linalg.norm(q) != 0:
        return H + (p@np.transpose(p))/(np.transpose(p)@q) - (H@q@np.transpose(q)@H)/(np.transpose(q)@H@q)
    else: 
        return H

In [222]:
def met_quase_newton_DFP(x,tol,maxiter,gama,n):
    x = np.array(x)
    m = 0
    k =0
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        H = -1*DFP(x,x_antigo,H)
        m = m + 1
        k = k + ar[1]
    return [x,m,k] 

In [223]:
met_quase_newton_DFP([[0.5],[0.5]],0.01,1000,0.8,0.25) 

[array([[0.67556966],
        [0.67556966]]),
 7,
 598]

In [232]:
met_quase_newton_DFP([[-1],[-1]],0.01,1000,0.8,0.25) 

[array([[-0.00021052],
        [-0.00021052]]),
 10,
 826]

#### BFGS

In [228]:
def BFGS(x_atual,x_antigo,H):
    x_atual = np.array(x_atual)
    x_antigo = np.array(x_antigo)
    p = x_atual - x_antigo
    q = Gradiente(x_atual)- Gradiente(x_antigo)
    if np.traspose(q)@p:
        return H + (1 + (np.transpose(q)@H@q)/np.transpose(p)@q)*((p@np.transpose(p))/(np.transpose(q)@q)) - (p@np.transpose(q)@H + H@q@np.transpose(p))/(np.transpose(p)@q)
    else: 
        return H

In [229]:
def met_quase_newton_BFGS(x,tol,maxiter,gama,n):
    x = np.array(x)
    m = 0
    k =0
    H = np.identity(2)
    while np.linalg.norm(Gradiente(x)) > tol and m < maxiter:
        x_antigo = x
        d = -1*H@Gradiente(x)
        ar = Armijo(x,gama,d,n)
        t = ar[0]
        x = x + t*d
        H = -1*BFGS(x,x_antigo,H)
        m = m + 1
        k = k + ar[1]
    return [x,m,k] 

In [230]:
met_quase_newton_BFGS([[0.5],[0.5]],0.01,1000,0.8,0.25) 

C:\Users\yfrom\AppData\Local\Temp\ipykernel_17260\3196596769.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 3**(1/(x1+x2))+x1**2+x2**2


[array([[-0.00116829],
        [-0.00116829]]),
 7,
 531]

In [231]:
met_quase_newton_BFGS([[-1],[-1]],0.01,1000,0.8,0.25) 

[array([[-0.00031065],
        [-0.00031065]]),
 5,
 385]